# Neural Network Variational Monte Carlo

1 Dimensional, N bosons or N_up and N_down fermions. 

In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"]="platform"
os.environ["JAX_ENABLE_X64"]="false"


# os.environ["XLA_FLAGS"]="--xla_cpu_multi_thread_eigen=true intra_op_parallelism_threads=16 --xla_force_host_platform_device_count={}".format(multiprocessing.cpu_count())
os.environ["XLA_FLAGS"]="--xla_cpu_multi_thread_eigen=true intra_op_parallelism_threads=16"

from jax import numpy as jnp
from jax import random, jit, jacfwd, grad, vmap, jvp
from jax.nn import one_hot
from jax.lax import fori_loop
from jax.tree_util import tree_map
from flax import linen as nn
import numpy as np
import time
from tqdm import trange
from functools import partial
import jax.example_libraries.optimizers as jax_opt
import plotly.graph_objects as go
import mc # sampler

In [50]:
bosonic = True
if bosonic:
    N = 5
else:
    N_up = 2
    N_down = 1

g = .4
omega = 1
m = 1
harmonic_omega = 1
sigma = -m*harmonic_omega*g/2
C = 10


def astra_energy():
    return (N * harmonic_omega)/2 - m * g**2  * (N*(N**2 - 1))/(24)

true_energy = astra_energy()

# true_energy = .5 * N

print("True energy: ", true_energy)
# compute a 1% bound for the true energy
true_energy_lower = true_energy * 0.99
true_energy_upper = true_energy * 1.01



total_energy = []
total_uncerts = []

fig = go.FigureWidget()

True energy:  1.6999999999999997


In [51]:

class MLP(nn.Module):
    features: list[int]  # e.g. [64, 64, 1]

    @nn.compact
    def __call__(self, x):
        for feat in self.features[:-1]:
            x = nn.Dense(feat)(x)
            x = nn.celu(x)
        x = nn.Dense(self.features[-1])(x)
        return x

from typing import Sequence, Callable, Optional

class ResidualMLP(nn.Module):
    features: Sequence[int]          # hidden widths, e.g. [128, 128, 128]
    out_dim: int = 1
    act: Callable = nn.celu
    dropout: float = 0.0
    use_layernorm: bool = True

    @nn.compact
    def __call__(self, x, *, train: bool = False):
        # x: (N,) or (batch, N)
        h = x
        for w in self.features:
            y = nn.Dense(w)(h)
            if self.use_layernorm:
                y = nn.LayerNorm()(y)
            y = self.act(y)
            if self.dropout > 0:
                y = nn.Dropout(self.dropout)(y, deterministic=not train)

            y = nn.Dense(w)(y)  # second linear in the block
            if self.use_layernorm:
                y = nn.LayerNorm()(y)

            # residual connection (project if needed)
            if h.shape[-1] != w:
                h = nn.Dense(w)(h)
            h = self.act(h + y)

        return nn.Dense(self.out_dim)(h).squeeze(-1)




@jit
def transform(coords):
    x = coords / C                                  # (N,)
    powers = jnp.cumprod(jnp.broadcast_to(x, (x.shape[0], x.shape[0])), axis=0)
    return jnp.sum(powers, axis=1)

def create_model(rng_key, N, features):
    model = MLP(features=features)
    # model = ResidualMLP(features=features[:-1], out_dim=features[-1], dropout=0.0, use_layernorm=False)
    dummy_x = jnp.ones((N,))          # N inputs of length d → shape (N,d)
    params = model.init(rng_key, dummy_x)["params"]
    return model, params


In [52]:
model, params = create_model(random.PRNGKey(int(time.time())), N, [50, 100, 100, 100, 100,1])
print(model)
# print(params)

MLP(
    # attributes
    features = [50, 100, 100, 100, 100, 1]
)


In [53]:

@jit
def A(x, params):
    """Neural network output, A(x) given a set of parameters"""
    return model.apply({'params': params}, transform(x)).squeeze() + omega * jnp.sum(x**2.)


@jit
def psi(x, params):
    """Wavefunction, psi(x) given a set of parameters"""
    return jnp.exp(-A(x, params))


print(psi(jnp.ones((N,)), params)) 

0.0073917126


In [54]:
mcs = mc.Sampler(psi, (N, 1))

In [55]:
# s,sp, ar = mcs.sample_parallel(params, 1000000//16, 1000, 3, .7, jnp.zeros((N,)), num_chains=16)
# print("num samples:", s.shape[0])
# print(ar)

In [56]:
s,sp, ar = mcs.sample_batched(params, 10000//16, 1000, 3, .3, jnp.zeros((N,)), num_chains=16, key=random.PRNGKey(int(time.time())))
print("num samples:", s.shape[0])
print(ar)
print(s[0])
print(sp[0])
print(s[:,0])
print(sp[:,-1])

num samples: 10000
0.70739126
[-0.4534186   0.4511728   0.14964408  0.47998768 -0.13952422]
[-0.4534186   0.4511728   0.14964408  0.47998768 -0.4534186 ]
[-0.4534186  -0.5534365  -0.27984267 ...  0.07559907  0.07061088
 -0.10250926]
[-0.4534186  -0.5534365  -0.27984267 ...  0.07559907  0.07061088
 -0.10250926]


In [57]:
# derivative of the wavefunction with respect to the parameters
dnn_dtheta = jit(grad(psi, 1))
vdnn_dtheta = jit(vmap(dnn_dtheta, in_axes=(0, None), out_axes=0))



grad_psi_x = jit(grad(psi, 0))  # returns (N,)
@partial(jit, static_argnames=("N",))
def laplacian_psi_jvp(x, params, *, N):
    def g(x_):
        return grad_psi_x(x_, params)

    def body(i, acc):
        ei = one_hot(i, N, dtype=x.dtype)
        _, dg = jvp(g, (x,), (ei,))
        return acc + dg[i]

    return fori_loop(0, N, body, jnp.array(0.0, x.dtype))



if bosonic: 
        num_interactions = N * (N - 1) // 2
else:
    num_interactions = N_up * N_down

dA_dtheta = jit(grad(A, 1))
vdA_dtheta = vmap(dA_dtheta, in_axes=(0, None), out_axes=0)

dA_dx = jit(grad(A, 0))
vdA_dx = vmap(dA_dx, in_axes=(0, None), out_axes=0)

A_hessian = jacfwd(jit(grad(A, 0)), 0)
@jit
def d2A_dx2(coords, params):
    return jnp.diag(A_hessian(coords, params))





@jit
def sigma_potential(x):
    # pairwise |xi-xj|
    diffs = jnp.abs(x[:, None] - x[None, :])         # (N,N)
    return sigma * jnp.sum(jnp.triu(diffs, k=1))     # sum_{i<j}


@jit
def Es_nodelta(coords, params):
    # return - (1/2) * (1/ psi(coords, params)) * jnp.sum(ddpsi(coords, params)) + (1/2) * jnp.sum(coords**2) 
    return - (1/2) * (1/ psi(coords, params)) * laplacian_psi_jvp(coords, params, N=N) + (1/2) * jnp.sum(coords**2) + sigma_potential(coords)
    # return (1/2) * (jnp.sum(d2A_dx2(coords, params)) - jnp.sum(dA_dx(coords, params)**2)) + (1/2) * jnp.sum(coords**2)  + sigma_potential(coords)

vEs_nodelta = vmap(Es_nodelta, in_axes=(0,None), out_axes=0)


@jit
def Es_delta(coords, coords_prime, params, alpha, g):
    return num_interactions * g * (psi(coords_prime, params)**2)/(psi(coords, params)**2) * (1/(jnp.sqrt(jnp.pi)*alpha))*jnp.exp(-(coords[-1]/alpha)**2)

vEs_delta = vmap(Es_delta, in_axes=(0,0, None, None, None), out_axes=0)

def pytree_scalar_mult(scalar, pytree):
    return tree_map(lambda x: scalar * x, pytree)

def pytree_add(a, b):
    return tree_map(lambda x, y: x + y, a, b)

@jit
def tree_mean(pytree_batched):
    return tree_map(lambda x: jnp.mean(x, axis=0), pytree_batched)

def tree_zeros_like(pytree):
    return tree_map(jnp.zeros_like, pytree)


@jit
def gradient_comp(coords, coords_prime, params, es_nodelta, energy_calc, es_delta):
    return pytree_add(pytree_scalar_mult(2/psi(coords, params) * (es_nodelta - energy_calc), dnn_dtheta(coords, params)), pytree_scalar_mult(2 * es_delta / psi(coords_prime, params), dnn_dtheta(coords_prime, params)))

vgradient_comp = vmap(gradient_comp, in_axes=(0, 0, None, 0, None, 0), out_axes=0)



@partial(jit, static_argnames=("chunk_size","g"))
def compute_stats_and_grad(samples, samples_prime, params, g, *, chunk_size=256):
    n = samples.shape[0]

    # alpha
    ys = samples[:, -1]
    maxabs = jnp.max(jnp.abs(ys))
    alpha = maxabs / jnp.sqrt(-jnp.log(jnp.sqrt(jnp.pi) * 1e-10))

    # energies
    e0 = vEs_nodelta(samples, params)
    ed = vEs_delta(samples, samples_prime, params, alpha, g)
    e = e0 + ed

    mean_e = jnp.mean(e)
    var_e = jnp.mean(e * e) - mean_e * mean_e
    uncert = jnp.sqrt(jnp.maximum(var_e, 0.0)) / jnp.sqrt(n)

    # chunks
    nb = (n + chunk_size - 1) // chunk_size

    grad0 = tree_zeros_like(dnn_dtheta(samples[0], params))

    def body(i, grad_sum):
        start = i * chunk_size
        idx = start + jnp.arange(chunk_size)          # (chunk_size,)
        mask = (idx < n).astype(samples.dtype)        # (chunk_size,)

        idx = jnp.minimum(idx, n - 1)                 # clamp for safe gather

        s_chunk  = samples[idx, :]
        sp_chunk = samples_prime[idx, :]
        e0_chunk = e0[idx]
        ed_chunk = ed[idx]

        grads = vgradient_comp(s_chunk, sp_chunk, params, e0_chunk, mean_e, ed_chunk)

        # multiply each leaf by mask on leading axis, then sum over axis 0
        def masked_sum(x):
            # x has leading dimension chunk_size
            reshape = (chunk_size,) + (1,) * (x.ndim - 1)
            return jnp.sum(x * mask.reshape(reshape), axis=0)

        grads_sum = tree_map(masked_sum, grads)
        return pytree_add(grad_sum, grads_sum)

    grad_sum = fori_loop(0, nb, body, grad0)
    mean_grad = pytree_scalar_mult(1.0 / n, grad_sum)

    return mean_grad, mean_e, uncert


def gradient(params, g, num_samples=10**3, thermal=200, skip=5, variation_size=1.0, sampling="serial", chunk_size=256):
    if sampling == "serial":
        samples, samples_prime, _ = mcs.sample(params, num_samples, thermal, skip, variation_size, jnp.zeros((N,)))
    elif sampling == "parallel":
        n_chains = 16
        samples, samples_prime, _ = mcs.sample_batched(params, num_samples//n_chains, thermal, skip, variation_size, jnp.zeros((N,)), num_chains=n_chains, key=random.PRNGKey(int(time.time())))

    return compute_stats_and_grad(samples, samples_prime, params, g, chunk_size=chunk_size)


def step(params, opt_state, step_num, num_samples, thermal, skip, variation_size, g, sampling, chunk_size):
    """
    One optimization step.
    - params: current parameters pytree
    - opt_state: optimizer state (must be carried across steps)
    Returns: (new_params, new_opt_state, energy, uncert)
    """
    grad, energy, uncert = gradient(
        params,
        g,
        num_samples=num_samples,
        thermal=thermal,
        skip=skip,
        variation_size=variation_size,
        sampling=sampling,
        chunk_size=chunk_size,
    )
    new_opt_state = opt_update(step_num, grad, opt_state)
    new_params = get_params(new_opt_state)

    return new_params, new_opt_state, energy, uncert

def update_plot(total_energy, total_uncerts):
    with fig.batch_update():
        fig.data[0].x = np.arange(len(total_energy))
        fig.data[0].y = total_energy
        fig.data[0].error_y.array = total_uncerts

        n = len(total_energy)
        for i in [1, 2, 3]:
            fig.data[i].x = [0, n]
              

def train(params, iterations, num_samples, thermal, skip, variation_size, g, sampling="serial", chunk_size=256, energy_storage=total_energy, uncert_storage=total_uncerts):
    """
    Training loop.
    Returns: (hs, us, ns, final_params)
    """
    hs, us = [], []
    ns = np.arange(iterations)

    # Initialize optimizer state ONCE
    opt_state = opt_init(params)

    pbar = trange(iterations, desc="", leave=True)
    old_params = params

    for step_num in pbar:
        new_params, opt_state, energy, uncert = step(
            old_params,
            opt_state,
            step_num,
            num_samples,
            thermal,
            skip,
            variation_size,
            g,
            sampling=sampling,
            chunk_size=chunk_size,
        )
        

        hs.append(energy)
        us.append(uncert)
        old_params = new_params

        energy_storage.append(energy)
        uncert_storage.append(uncert)
        update_plot(energy_storage, uncert_storage)

        pbar.set_description(f"Energy = {energy}", refresh=True)

        # Use jnp.isnan if energy is a JAX scalar; np.isnan is OK if it's a Python float
        if np.isnan(np.asarray(energy)):
            print("NaN encountered, stopping...")
            #TODO: backtrack to previous params, try again?
            break

    return hs, us, ns, old_params



In [58]:



energy_trace = fig.add_scatter(
    x=[],
    y=[],
    error_y=dict(type='data', array=[], visible=True),
    mode='lines+markers',
    name='Energy vs Iteration'
)


# add a horizontal line for the true energy
fig.add_trace(go.Scatter(
    x=[0, len(total_energy)],
    y=[true_energy, true_energy],
    mode='lines',
    name='True Energy',
    line=dict(dash='dash', color='red')
))

fig.add_trace(go.Scatter(
    x=[0, len(total_energy)],
    y=[true_energy_lower, true_energy_lower],
    mode='lines',
    name='True Energy Lower Bound',
    line=dict(dash='dot', color='green')
))

fig.add_trace(go.Scatter(
    x=[0, len(total_energy)],
    y=[true_energy_upper, true_energy_upper],
    mode='lines',
    name='True Energy Upper Bound',
    line=dict(dash='dot', color='green')
))

fig.update_layout(
    title='VMC Energy Convergence',
    xaxis_title='Iteration',
    yaxis_title='Energy',
    template='plotly_dark'
)

display(fig)


FigureWidget({
    'data': [{'error_y': {'array': [], 'type': 'data', 'visible': True},
              'mode': 'lines+markers',
              'name': 'Energy vs Iteration',
              'type': 'scatter',
              'uid': '2f105044-7209-4fda-8a34-f272cd1d6eae',
              'x': [],
              'y': []},
             {'line': {'color': 'red', 'dash': 'dash'},
              'mode': 'lines',
              'name': 'True Energy',
              'type': 'scatter',
              'uid': 'a54586e8-8949-4aff-a3ed-5a00ee9b271d',
              'x': [0, 0],
              'y': [1.6999999999999997, 1.6999999999999997]},
             {'line': {'color': 'green', 'dash': 'dot'},
              'mode': 'lines',
              'name': 'True Energy Lower Bound',
              'type': 'scatter',
              'uid': '6b73fba4-2c8b-4764-9c8a-61ced353654b',
              'x': [0, 0],
              'y': [1.6829999999999998, 1.6829999999999998]},
             {'line': {'color': 'green', 'dash': 'dot'},
   

In [59]:
opt_init, opt_update, get_params = jax_opt.adam(10 ** (-3))
# params, N_steps, N_samples, thermalization, skip, variation_size, g, sampling_type, gradient chunk size
resultsa = train(params, 400, 20000, 500, 2, 0.3, g, sampling="parallel", chunk_size=2000)


Energy = 1.8311909437179565: 100%|██████████| 400/400 [03:12<00:00,  2.08it/s]


In [60]:
opt_init, opt_update, get_params = jax_opt.adam(10 ** (-4))

resultsb = train(resultsa[3], 100, 50000, 1000, 4, 0.3, g, sampling="parallel", chunk_size=5000)

Energy = 1.7719908952713013: 100%|██████████| 100/100 [02:02<00:00,  1.23s/it]


In [ ]:
resultsc = train(resultsb[3], 200, 100000, 1000, 2, 0.7, g, sampling="parallel", chunk_size=2000)

In [ ]:
# opt_init, opt_update, get_params = jax_opt.adam(10 ** (-4))

resultsd = train(resultsc[3], 200, 100000, 2000, 3, 0.7, g, sampling="parallel", chunk_size=3000)

In [ ]:
res = resultsa[0] + resultsb[0] + resultsc[0] + resultsd[0]
res_uncerts = resultsa[1] + resultsb[1] + resultsc[1] + resultsd[1]

In [ ]:

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=np.arange(len(res)),
    y=res,
    error_y=dict(
        type='data',
        array=res_uncerts,
        visible=True
    ),
    mode='lines+markers',
    name='Energy vs Iteration'
))

# add a horizontal line for the true energy
fig.add_trace(go.Scatter(
    x=[0, len(res)],
    y=[true_energy, true_energy],
    mode='lines',
    name='True Energy',
    line=dict(dash='dash', color='red')
))

fig.add_trace(go.Scatter(
    x=[0, len(res)],
    y=[true_energy_lower, true_energy_lower],
    mode='lines',
    name='True Energy Lower Bound',
    line=dict(dash='dot', color='green')
))

fig.add_trace(go.Scatter(
    x=[0, len(res)],
    y=[true_energy_upper, true_energy_upper],
    mode='lines',
    name='True Energy Upper Bound',
    line=dict(dash='dot', color='green')
))

fig.update_layout(
    title='VMC Energy Convergence',
    xaxis_title='Iteration Number',
    yaxis_title='Energy',
    template='plotly_dark'
)
fig.show()

In [ ]:
test = jnp.array([1.0, 2.0, 4.0])
test2 = jnp.array([2.0, 1.0, 4.0])
print(psi(test, params))
print(psi(test2, params))

In [ ]:
samples, samples_prime, ar = mcs.sample_batched(resultsa[3], 1000000//16, 1000, 3, .7, jnp.zeros((N,)), num_chains=16, key=random.PRNGKey(int(time.time())))

In [ ]:
# histogram of the samples for each particle
import matplotlib.pyplot as plt
for i in range(N):
    plt.figure()
    plt.hist(samples_prime[:, i], bins=100, density=True)
    plt.title(f'Histogram of samples for particle {i+1}')
    plt.xlabel('Position')
    plt.ylabel('Density')
    plt.grid()
    plt.show()
